In [296]:
print("Loading modules, classes and methods...")
import multiprocessing
from unidecode import unidecode
from joblib import Parallel, delayed
import numpy as np
import string
import pandas as pd
import time
import os
import numpy as np
import re
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchvision
import pickle
from unicodedata import normalize
from keras.preprocessing.text import Tokenizer
from torchvision import transforms, datasets
from tqdm import tqdm
from time import sleep
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import textclassification as tc

Loading modules, classes and methods...


In [297]:
print("Loading data...")

STARTING = 0
ENDING = 2000000
valid = np.load("../light_data/new_classes.npy")
prevalences = pd.read_csv("../light_data/prevalencia_e_scores_com_siglas.csv")
dicionarioCsv = pd.read_csv("../light_data/DicionarioECG_completo.csv", index_col = 0)
siglasCsv = pd.read_csv("../light_data/DicionarioECG_Siglas.csv", index_col = 0)
db = pd.read_csv("../../data/DATA_LAUDOS_TEXTO_formato1", sep = ";")


print("Cutting slice. From {} to {}...".format(STARTING, ENDING))
db = db[STARTING:ENDING]
db = db.reset_index()

Loading data...
Cutting slice. From 0 to 2000000...


In [298]:
import pandas as pd

dicionarioCsv = pd.read_csv("../light_data/DicionarioECG_completo.csv", index_col = 0)
dicionario = {}
for row in dicionarioCsv.itertuples():
    aux = []
    for diag in row:
        if type(diag) is str: aux.append(" " + diag + " ")
    dicionario[row[0]] = aux

In [299]:
print("Preparing texts...")
texts = [" " + tc.clean_text(text) + " " for text in db["CONTEUDO"]]

siglas = {}
for row in siglasCsv.itertuples():
    aux = []
    for diag in row:
        if type(diag) is str: aux.append(" " + diag + " ")
    siglas[row[0]] = aux
siglas[11].append(" bav2:1 ")
    
dicionario = {}
for row in dicionarioCsv.itertuples():
    aux = []
    for diag in row:
        if type(diag) is str: aux.append(" " + diag + " ")
    dicionario[row[0]] = aux

Preparing texts...


In [300]:
result = np.load("output/score_values.npy")

In [240]:
import math
print("Calculating thresholds and creating binary arrays...")
scores_thresh = []
y_bin = np.zeros((len(result),valid[:,1].sum()))
nClass = 0
for i in range(len(result[0])):
    if(not valid[i][1]): continue
    print("Calculating for", prevalences.loc[i]["Diagnostico"])
    temp_result       = result[:,i]
    temp_result = np.sort(temp_result, kind = 'mergesort')
    ocurrences     = len(result) * prevalences.loc[i]["Prevalencia"]
    ocurrences     = math.ceil(ocurrences)
    scores_thresh.append(temp_result[-ocurrences])
#     for j in range(len(result)):
#         if result[j][i] >= scores_thresh[-1]: y_bin[j][nClass] = 1
    print("Threshold =",temp_result[-ocurrences])
    nClass += 1

# np.save("output/score_bin.npy", y_bin)
# np.save("output/score_thresholds.npy", scores_thresh)

Calculating thresholds and creating binary arrays...
Calculating for área_eletricamente_inativa
Threshold = 82.143
Calculating for Bloqueio_de_ramo_direito
Threshold = 88.889
Calculating for Bloqueio_de_ramo_esquerdo
Threshold = 89.286
Calculating for Sobrecarga_ventricular_esquerda_(critérios_de_Romhilt-Estes)
Threshold = 93.939
Calculating for Fibrilação_atrial
Threshold = 78.947
Calculating for Flutter_atrial
Threshold = 87.5
Calculating for Bloqueio_atrioventricular_de_2°_grau_Mobitz_I
Threshold = 75.0
Calculating for Pré-excitação_ventricular_tipo_Wolff-Parkinson-White
Threshold = 77.778
Calculating for Sistema_de_estimulação_cardíaca_normofuncionante
Threshold = 59.459
Calculating for Taquicardia_atrial_multifocal
Threshold = 96.774
Calculating for Taquicardia_supraventricular
Threshold = 67.742
Calculating for Alterações_primárias_da_repolarização_ventricular
Threshold = 73.077
Calculating for Extrassístoles_supraventriculares
Threshold = 61.111
Calculating for Extrassístoles_ve

In [301]:
scores_thresh = np.load("output/score_thresholds.npy")

In [246]:
dicionarioCsv = pd.read_csv("../light_data/DicionarioECG_completo.csv", index_col = 0)
scores_thresh = np.load("scores_threshold.npy")
valid = np.load("../light_data/new_classes.npy")

In [227]:
y_bin = np.load("output/19_Jul_2020/score_bin.npy")

In [233]:
y_bin[:,9].sum()

66.0

In [238]:
shorter = "taquicardia atrial multifocal#"
shorter = shorter[:-1]
shorter

'taquicardia atrial multifocal'

In [245]:
trans = {}
for i in range(35):
    aux = 0
    for z in range(74):
        aux += valid[z][1]
        if(aux == (i+1)): 
            trans[i] = z
            break

In [320]:
scores_old = np.load("scores_thresholds_old.npy")

In [617]:
for i in range(35):
    print(i, " ", dicionario[trans[i]][0], "->", scores_thresh[i])

0    área eletricamente inativa  -> 80.0
1    Bloqueio de ramo direito#  -> 88.0
2    Bloqueio de ramo esquerdo#  -> 88.0
3    Sobrecarga ventricular esquerda (critérios de Romhilt-Estes)  -> 90.0
4    Fibrilação atrial  -> 89.0
5    Flutter atrial  -> 80.0
6    Bloqueio atrioventricular de 2° grau Mobitz I#  -> 85.0
7    Pré-excitação ventricular tipo Wolff-Parkinson-White  -> 80.0
8    Sistema de estimulação cardíaca normofuncionante  -> 62.162
9    Taquicardia atrial multifocal#  -> 90.0
10    Taquicardia supraventricular#  -> 80.0
11    Alterações primárias da repolarização ventricular#  -> 90.0
12    Extrassístoles supraventriculares#  -> 80.0
13    Extrassístoles ventriculares#  -> 80.0
14    Bradicardia sinusal#  -> 85.0
15    ECG dentro dos limites da normalidade para idade e sexo  -> 80.0
16    Alterações inespecíficas da repolarização ventricular#  -> 78.0
17    Alterações secundárias da repolarização ventricular#  -> 85.0
18    Arritmia sinusal  -> 80.0
19    Ausência de sin

In [133]:
prevalences.loc[trans[c]]

Unnamed: 0                                                    29
Diagnostico    Alterações_secundárias_da_repolarização_ventri...
Prevalencia                                          9.86585e-06
Score                                                        100
Name: 28, dtype: object

In [294]:
result[:,9].sum() / 2000000

36.14832529550001

In [609]:
scores_thresh[c] = 85

In [308]:
scores_thresh[0]

58.333

In [612]:
c = 35
i = 0
a = 0
print(dicionario[trans[c]][0])
print(scores_thresh[c])
print()
while(1):
    if result[i][trans[c]] >= scores_thresh[c] and result[i][trans[c]] <= scores_thresh[c]+100:
        if(a < 11):
            print("Id:",i)
            print("Score:",result[i][trans[c]])
            print(texts[i])
            print()
        a += 1
    i += 1
    if(i == 2e6):
        print()
        print("Should be",i," -> ", i*prevalences["Prevalencia"][trans[c]])
        print()
        print("We got",i, " -> ", a)
        break

KeyError: 35

In [613]:
np.save("scores_threshold.npy", np.array(scores_thresh))

In [614]:
a = np.load("output/scores_thresholds.npy")